In [ ]:
!pip install google-search-results

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for google-search-results: filename=google_search_results-2.4.1-py3-none-any.whl size=25789 sha256=3b954451dbdfba64d8b0407403d7b3bf6815ca6c0c8b52336bb0a88bb6e7a12a
  Stored in directory: /root/.cache/pip/wheels/82/a3/c5/364155118f298722dff2f79ae4dd7c91e92b433ad36d6f7e0e
Successfully built google-search-results


In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from time import time
import seaborn as sns
from datetime import datetime
import re
import matplotlib.pyplot as plt

### Intro

Different types of serp answer box are:
1. Calculator  
2. weather
3. finance
4. population
5. converter
6. flights
7. hotels
8. dictionary
9. list
10. organic_result
11. Multiple answers
12. translation result
13. directions






Out of these we might need:
8. dictionary
9. list
10. organic_result
11. Multiple answers
12. translation result



In [ ]:
from serpapi import GoogleSearch

params = {
  "q": "ethylene forms ethylene chlorohydrin by the action of",
  "hl": "en",
  "gl": "us",
  "api_key": "224556e81683d597bbbf5008180f729530ad4aa94439813754458f25f2ab826d"
}

search = GoogleSearch(params)
results = search.get_dict()
answer_box = results["answer_box"]

https://serpapi.com/search


In [ ]:
answer_box['type']

'organic_result'

In [ ]:
answer_box

{'about_page_link': 'https://www.google.com/search?q=About+https://www.vedantu.com/question-answer/ethylene-forms-ethylene-chlorohydrin-by-the-class-12-chemistry-cbse-5f625c5a01faef2daa4dfc3e&tbm=ilp&ilps=ADNMCi2D5xNDr0bR51v-WM3hzLuEPs39dA',
 'about_this_result': {'keywords': ['ethylene',
   'forms',
   'chlorohydrin',
   'action',
   'of'],
  'languages': ['English'],
  'regions': ['the United States'],
  'source': {'description': 'Vedantu is an Indian online tutoring platform launched in 2014 based in Bengaluru, India.',
   'icon': 'https://serpapi.com/searches/62b0fb1a3c3fb29767325760/images/52dd021a7ffd5758adb0d9ddb6ad14823b6b88984cfca03661f089c891d7cb7b54194fbaf3330050619d9ca806dd9869.png'}},
 'displayed_link': 'https://www.vedantu.com › question-answer › ethylene-fo...',
 'link': 'https://www.vedantu.com/question-answer/ethylene-forms-ethylene-chlorohydrin-by-the-class-12-chemistry-cbse-5f625c5a01faef2daa4dfc3e#:~:text=From%20the%20above%20observation%2C%20ethylene,answer%20is%20

In [ ]:
answer_box['snippet']

'From the above observation, ethylene forms ethylene chlorohydrins by the action of a solution of chlorine gas in water. So, the correct answer is “Option C”.'

In [ ]:
questions=  [
             "Second stage in the preparation of final accounts is", # No answer_box. Organic result in position 1 gives good result => organic_result=>snippet
             "Distinguish between Debit Note and Credit Note ", # answer_box => snippet. Same as answer_box => title
             "Minimum F S F bond angle present in ", # => answer_box => snippet
             "Ultraviolet radiation is absorbed by ", # => here organic_result better than answer_box
             "What is electrostatic shielding ", # answer_box => snippet
             "Which of the following letters do not surface lateral inversion", # => answer_box=> snippet
             "The capacity of a parallel plate condenser is C Its capacity when the separation between the plates is halved will be " # organic_result => snippet
            ]   

Based on the above results, I think that porbably a good method would be to first add the answer snippets for those questions that have a answer_box. If they don't then add the snippets from the first organic result.

### Loading the data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("drive/MyDrive/Extramarks Project/Dataset versions/JEE-with-answers-without-numericals.csv")
df = df.drop("Unnamed: 0", axis = 1)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
df['syllabus_id'].nunique()

6548

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df = df[:100]

In [ ]:
df.to_csv("drive/MyDrive/Extramarks Project/Dataset versions/DEMO-JEE-with-answers-without-numericals.csv")

In [ ]:
df

,question_id,question,syllabus_id,answer,question_type
0,1521080,The devices that send data and instructions in...,213057,Input devices,Single Choice Question
1,1789722,How is the government formed at the state level,672736,Each state in India has a Legislative Assembly...,Subjective Question
2,1288273,Describe the action of relays,417128,A relay is a switch operated by an electromagn...,Subjective Question
3,1462603,Put this book _______,5584,there,Single Choice Question
4,1107159,What other extraordinary things happen at the ...,453,Usually Griffin was quite particular about not...,Subjective Question
...,...,...,...,...,...
95,1757315,After indigo cultivation collapsed in Bengal w...,667940,Bihar,Single Choice Question
96,1800031,Who was the last Mughal Emperor in India,667943,Bahadur Shah Zafar was the last of the Mughal ...,Subjective Question
97,834506,What is the chief mode of transport system ava...,86886,water transport,Single Choice Question
98,489682,The things ____________________,4709,were hidden,Single Choice Question


Let's see how many questions have no answer 

In [ ]:
print("question_nans: ", df['question'].isna().sum())
print("answer_nans: ", df['answer'].isna().sum())
print("question_id_nans: ", df['question_id'].isna().sum())
print("syllabus_id_nans: ", df['syllabus_id'].isna().sum())
print("question_type_nans: ", df['question_type'].isna().sum())

question_nans:  0
answer_nans:  581
question_id_nans:  0
syllabus_id_nans:  0
question_type_nans:  0


In [ ]:
df = df.dropna()

Then some of the questions have images as answers. That doesn't suit us as well

In [ ]:
print(df.iloc[3]['answer'])
print(df.iloc[4]['answer'])

stryde images 2022 04 11 20220411016213001649675720 png 
stryde images 2022 04 11 20220411969337001649675598 png x 6 y 6 z 1 y z x 6 1 6 0 


In [ ]:
# df = df.sample(frac=1).reset_index(drop=True)
# df = df[:40000]

In [ ]:
# patternDigit = re.compile(r'.*(stryde images|ckfinder userfiles).*') 
#regex useless here

In [ ]:
image_answers_count= 0
list_image_answers = []


for index, i in df.iterrows():
  if index%20000 == 0:
    print(index)  
  ans =  i['answer'].split()
  # if len(patternDigit.findall(i['answer'])) > 0:
  if "stryde" in ans and "images" in ans:
    image_answers_count+=1
    list_image_answers.append(index)

  if "ckfinder" in ans and "userfiles" in ans :
    image_answers_count+=1
    list_image_answers.append(index)

print("questions with an image link in the answer: ", image_answers_count)

0
20000
40000
60000
80000
100000
120000
questions with an image link in the answer:  2727


About 2% of the dataframe has images as answers

In [ ]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("DONE. GMT(00:00) time is :", current_time)

DONE. GMT(00:00) time is : 11:48:11


###Results of the obje2subj repo by Venktesh 

Taking a look at some of the examples that the repo provides

**Question 1**\
 Q. The words like and as usually appear in a  
 A. simile


**Most similar:**\
Question #1: What words can be used in a simile? - Confidence - 76%\
Question #2: What do you call like and as in simile? - Confidence - 75%\
Question #3: What is the simile of as like as? - Confidence - 70%

**Question 2**\
 Q. The devices that send data and instructions into a computer are known as ___________   
 A. Input devices 


**Most similar:**\
Question #1: Question #1: What are the devices that send data and instructions into a computer called? - Confidence - 86%\
Question #2: Question #2: What are devices that send data and instructions into a computer called?- 86%\
Question #3: Question #3: What is the input devices of computer? - Confidence - 82%

**Question 3**\
 Q. Choose the correct option\
 A. Yesterday the police arrested a notorious criminal in the city in connection with the shootout \


**Most similar:**\
Resulted in a KeyError

**Question 4**\
 Q. Metal hydride on treatment with water gives 
 A. Hydrogen


**Most similar:**\
Question #1: What substance is treated with metal hydride?\
Question #2: Do metal hydrides react with water?\
Question #3: How do metal hydrides store hydrogen?\

**Question 5**\
 Q. Corn with hard kernels that swell up and burst open when heated ____ corn  \
 A. Pop\


**Most similar:**\
Question #1: What causes a kernel of corn to pop? - 74\
Question #2: Can corn pop from heat? -73%\
Question #3: What do corn kernels do when heated? -70%\

**Question 6**\
 Q. Which one of the following esters cannot undergo Claisen self condensation \
 A. C6H5COOC2H5


Unable to resolve